In [2]:
''' 
Link to Documentation: https://github.com/ultralytics/yolov5

RUN IN TERMINAL BEFORE PROCEEDED IN THE 'fashion-forecast' DIRECTORY:
>   pip install ultralytics
>   git clone https://github.com/ultralytics/yolov5  
>   cd yolov5
>   pip install -r requirements.txt  
>   cd ..
'''

" \nLink to Documentation: https://github.com/ultralytics/yolov5\n\nRUN IN TERMINAL BEFORE PROCEEDED IN THE 'fashion-forecast' DIRECTORY:\n>   pip install ultralytics\n>   git clone https://github.com/ultralytics/yolov5  \n>   cd yolov5\n>   pip install -r requirements.txt  \n>   cd ..\n"

In [3]:
import torch
import numpy as np
import pandas as pd
import os
from PIL import Image

%matplotlib inline 
from matplotlib import pyplot as plt

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True) #force_reload=True

Using cache found in C:\Users\cardon/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-16 Python-3.8.5 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [5]:
'''
CODE TO CHANGE DEVICE:
model.cpu()  # CPU
model.cuda()  # GPU
model.to(device)  # i.e. device=torch.device(0)
'''

'\nCODE TO CHANGE DEVICE:\nmodel.cpu()  # CPU\nmodel.cuda()  # GPU\nmodel.to(device)  # i.e. device=torch.device(0)\n'

In [ ]:
img_list = []
img_names = []
directory = '../unpack-data/'
printout_size = 10000
seasons_only_csv = pd.read_csv('filtered_style_stats.csv')
# Only has memory for 40,000 images at a time
counter = 0
for img_name in os.listdir(directory):
    if counter == len(seasons_only_csv.index):
        print("Breaking the loop at ", counter)
        break
    if img_name not in seasons_only_csv.iloc[:, 0].values:
        continue
    if counter%printout_size == 0:
        print(f"Running... {(counter/len(seasons_only_csv.index))*100:.2f}% finished")
    img_path = os.path.join(directory, img_name)
    pil_img = Image.open(img_path)
    np_img = np.array(pil_img)
    #plt.imshow(np_img)             # used to display the images
    #plt.show()
    img_list.append(np_img)
    img_names.append(str(img_path))
    counter += 1
print(f'Finished with a set of {len(img_list)} images')

Running... 0.00% finished
Running... 0.36% finished
Running... 0.72% finished
Running... 1.07% finished
Running... 1.43% finished
Running... 1.79% finished
Running... 2.15% finished
Running... 2.51% finished
Running... 2.87% finished
Running... 3.22% finished
Running... 3.58% finished
Running... 3.94% finished
Running... 4.30% finished
Running... 4.66% finished
Running... 5.01% finished
Running... 5.37% finished
Running... 5.73% finished
Running... 6.09% finished
Running... 6.45% finished
Running... 6.81% finished
Running... 7.16% finished
Running... 7.52% finished
Running... 7.88% finished
Running... 8.24% finished
Running... 8.60% finished
Running... 8.95% finished
Running... 9.31% finished
Running... 9.67% finished
Running... 10.03% finished
Running... 10.39% finished
Running... 10.74% finished
Running... 11.10% finished
Running... 11.46% finished
Running... 11.82% finished
Running... 12.18% finished
Running... 12.54% finished
Running... 12.89% finished
Running... 13.25% finished
Ru

MemoryError: Unable to allocate 703. KiB for an array with shape (600, 400, 3) and data type uint8

279203
Running... 0.00% finished


In [11]:
results = model(img_list)  # inference
results.save()   # cropped detections dictionary
#results.show()

Saved 364 images to runs\detect\exp5


In [ ]:
detection_df = results.pandas().xyxy
print(len(detection_df))
'''
for idx, df in enumerate(detection_df):
    person_df = df[df['name']=='person']
    print(person_df.index)
'''

50


"\nfor idx, df in enumerate(detection_df):\n    person_df = df[df['name']=='person']\n    print(person_df.index)\n"

In [ ]:
"""Takes the current results and saves only the information from the maximum 'person' detections"""

cleaned_df_lst = []
for idx, df in enumerate(detection_df):
    person_df = df[df['name']=='person']
    if person_df.empty == True:
        continue
    if (len(person_df.index) == 1):
        row = person_df.iloc[0].tolist()
        row.append(idx)
        cleaned_df_lst.append(row)
        continue
    # multiple detections -> choose the highest confidence of person
    maxValueIndex = person_df['confidence'].idxmax()
    row = person_df.iloc[maxValueIndex].tolist()
    row.append(idx)
    cleaned_df_lst.append(row)

col_names = list(detection_df[0].columns) + ['df_idx']
cleaned_df = pd.DataFrame(cleaned_df_lst, columns = col_names)
TOTAL_IMG = len(cleaned_df.index)
print(TOTAL_IMG)


49


In [ ]:
# Used to crop image in a 1:1 aspect ratio no smaller than 256 pixels centered around the person detection
def crop_image(img, xmin, ymin, xmax, ymax): 
    max_dim = max((xmax-xmin), (ymax-ymin))
    img = img[ymin:ymax, xmin:xmax]
    img_square = np.ones([max_dim+1,max_dim+1], dtype=np.uint8)*127
    xmin, xmax, ymin, ymax = 0, (xmax-xmin), 0, (ymax-ymin)
    img_square[ymin:ymax, xmin:xmax] = img
    return img_square

In [ ]:
"""Takes the cleaned df data and crops each image to it's detected bounding box dimensions"""
directory = '../unpack-data/'
save_dir = '../crop-data/'
df_idx = 0
for idx, img_name in enumerate(os.listdir(directory)):
    if idx == TOTAL_IMG:
        break
    if idx not in cleaned_df['df_idx'].values:
        continue

    xmin, ymin, xmax, ymax = cleaned_df.loc[df_idx][['xmin', 'ymin', 'xmax', 'ymax']]
    #print(int(xmin), int(ymin), int(xmax), int(ymax))
    df_idx += 1
    img_path = os.path.join(directory, img_name)
    pil_img = Image.open(img_path).convert("L")
    np_img = np.array(pil_img)
    crop_img = crop_image(np_img, int(xmin), int(ymin), int(xmax), int(ymax))
    #plt.imshow(crop_img, cmap='gray')
    save_path = os.path.join(save_dir, (img_name+'.jpg'))
    save_im = Image.fromarray(crop_img)
    save_im.save(save_path)